# Quick and Dirty Machine Learning: Modeling Earthquake Damage

In this post, I would like to demonstrate a quick way to create a good performing machine learning model. It is often encouraging to get a good performing model fast, without major pre-processing and featuring engineering. After this, we can use model selection, hyper-parameter tweaking, and feature engineering to squeeze out a little more predictive performance. However, it is really encouraging to get a well-performing fast.

So let's get started! I will be using data from a Data Science competition to asses the model's performance. The contest is called <a href="https://www.drivendata.org/competitions/57/nepal-earthquake/">"Richter's Predictor: Modeling Earthquake Damage"</a> hosted by <a href="https://www.drivendata.org/" target="_blank">DrivenData</a>. The goal is to predict the damage to buildings caused by an earthquake. The amount of damage is divided into 3 categories:  (1) low, (2) medium, (3) almost complete destruction.

In the cell below, we load the train values, train labels, test values, and the sample submission file and display the first couple of rows.

In [1]:
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
train_labels = pd.read_csv("data/train_labels.csv").drop('building_id', axis=1)
train_values = pd.read_csv("data/train_values.csv").drop('building_id', axis=1)

test_values = pd.read_csv("data/test.csv").drop('building_id', axis=1)
sample_sub = pd.read_csv("data/submission_format.csv")

train_values.head(2)

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
1,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


## Distribution of classes

One of the first things you should when dealing with classification is to look at the distribution of classes. For this, I use pandas value_count capabilities to get the frequency distribution of the target labels. Not all target categories are equally present. 

Label 2 is the majority class, with 56% of the observations. This percentage will be our baseline score. We need to outperform this score for our model to add something valuable. Otherwise, we would be better of always predicting label 2. 

In [3]:
train_labels.damage_grade.value_counts(normalize=True).sort_index()

1    0.096408
2    0.568912
3    0.334680
Name: damage_grade, dtype: float64

## Preprocessing

The only preprocessing we'll do is to get the data ready for the machine learning algorithm.

### Missing data

Often machine learning models don't allow for missing values in our data. We should identify these values and treat them. In our specific case, we are lucky because we don't have any missing values. However, if we would encounter them, these values need to be replaced. 

A good starting point is replacing missing numerical values with the mean or median, and replacing missing categorical values with the most frequent (mode) value. Note that your method of imputing missing values can have an impact on the performance of your model.

In [4]:
train_values.isna().sum().sum(), test_values.isna().sum().sum()

(0, 0)

### Categorical features

Most of the time, machine learning models also don't allow for categorical variables. We need to identify these features and encode them. One important note is that there should be consistency between the numerical representation of these values in the features in the train and test set.  

This can be achieved by first concatenating the test data at the end of the train data. Secondly, we encode the categorical features, after which we split the data back into a train and test set. We now have a constant representation of these variables between our train and test set.

Label encoding is a good starting point. This method encodes features with a value between 0 and n_classes-1. This technique works best with ordinal features because it implies ordering. However, because this post is about getting a quick model done, we apply it to all categorical features. We use some assert statements to test if the shape of the original train and test set are the same as the encoded ones.

In [5]:
from sklearn.preprocessing import LabelEncoder

all_data = pd.concat([train_values, test_values])
categorical_features = all_data.select_dtypes('object').columns

label_encoder = LabelEncoder()
all_data[categorical_features] = all_data[categorical_features].apply(label_encoder.fit_transform)

train_enc = all_data[:len(train_values)]
test_enc = all_data[len(train_values):]

assert train_enc.shape == train_values.shape 
assert test_enc.shape == test_values.shape

train_enc.head(3)

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,6,487,12198,2,30,6,5,2,2,0,0,1,3,2,1,1,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0
1,8,900,2812,2,10,8,7,1,2,0,3,1,2,2,0,1,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0
2,21,363,8973,2,10,5,5,2,2,0,0,3,3,2,0,1,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0


## Cross-validation

Before training our model, we should create a validation dataset. This lets us internally test our model, before predicting the test data. We're dealing with a classification problem, with an uneven class distribution. Therefore we should us stratification when splitting our data into a train and validation sets. This ensures that we have the same class distribution in our train and validation set.

In [6]:
from sklearn.model_selection import train_test_split

X = train_enc
y = train_labels

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

## Modeling the Data

Now that this is all done, we can start training our machine learning model. We use a LightGBM, which is a gradient boosting framework that uses tree-based learning algorithms. According to the developers, it is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support for parallel and GPU learning.
* Capable of handling large-scale data.

A basic LightGBM model can be trained in the same way as you fit a scikit-learn machine learning model. We start with 200 estimators, max depth of 50 with 900 leaves. This model only takes about a minute to train.

In [7]:
%%time
import lightgbm as lgb
model = lgb.LGBMClassifier(random_state=1, n_estimators=200, max_depth=50, num_leaves=900)
model.fit(X_train, y_train)
preds = model.predict(X_valid)

CPU times: user 3min 7s, sys: 2.76 s, total: 3min 10s
Wall time: 51.2 s


## Evaluating the model

To evaluate the performance of our model, we use the **micro averaged f1 score**, the same as in the contest. Our model gives us an f1-micro averaged score of **0.7422**, which is quite good. Especially when you keep in mind that we did so little processing and feature engineering.

In [8]:
from sklearn.metrics import f1_score

score_model = f1_score(y_valid, preds, average='micro')
round(score_model, 4)

0.7422

The model does outperform the naive majority class which gets a micro f1-score of **0.5689**.

In [9]:
naive_pred = np.zeros((len(y_valid))) + 2
score_naive = f1_score(y_valid, naive_pred, average='micro')
round(score_naive, 4)

0.5689

## Training the full model and predicting on test data

We can now train on the full dataset, which also lets us train on the validation set. After this, we can make predictions on the test dataset.

In [10]:
%%time
model.fit(X, y)
test_preds = model.predict(test_enc)

CPU times: user 3min 45s, sys: 3.1 s, total: 3min 48s
Wall time: 1min 1s


We store the test dataset predictions in the damage_grade columns of the sample submission file and save it as a CSV.

In [11]:
sample_sub['damage_grade'] = test_preds
sample_sub.to_csv("prediction.csv", index=None)

## Conclusion

After submitting to the competition, we a score of **0.7446**. As of 2020-03-18, this ranks us in the top **6-7%** of the leaderboard. This post demonstrated that we can achieve quite good performing models with almost no preprocessing, by leveraging the power of the LightGBM machine learning framework.